In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from snmfem.experiments import perform_simulations, load_samples, print_results, load_data, run_experiment

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Running NMF

## Inputs

In [ ]:
dataset = "dataset_EDXS_small.json"
samples, k = load_samples(dataset)
n_samples = 5
samples = samples[:n_samples]



In [ ]:
# About initialization
# 'random': non-negative random matrices, scaled with: sqrt(X.mean() / n_components)
# 'nndsvd': Nonnegative Double Singular Value Decomposition (NNDSVD) initialization (better for sparseness)
# 'nndsvda': NNDSVD with zeros filled with the average of X (better when sparsity is not desired)
# 'nndsvdar' NNDSVD with zeros filled with small random values (generally faster, less accurate alternative to NNDSVDa for when sparsity is not desired)

# for me random was the best initialization...
default_params = {
    "n_components" : k,
    "tol" : 1e-3,
    "max_iter" : 1000,
    "init" : "random",
    "random_state" : 1,
    "verbose" : 0
    }

params_snmf = {
    "force_simplex" : True,
    "skip_G" : False,
    "mu": 0
}

params_evalution = {
    "u" : True,
}

# All parameters are contained here
exp_list = [ 
    {"name": "scikit L2", "method": "SKNMF", "params": {**default_params, }},
    {"name": "scikit KL", "method": "SKNMF", "params": {**default_params, "beta_loss" : "kullback-leibler", "solver":"mu"}},
    {"name": "snmfem no G", "method": "NMF", "params": {**default_params, "force_simplex" : True, "skip_G" : True, "mu": 0}},
    {"name": "snmfem", "method": "NMF", "params": {**default_params, **params_snmf}},
    {"name": "snmfem smooth 3", "method": "SmoothNMF", "params": {**default_params, **params_snmf, "lambda_L" : 3.0}},
    {"name": "snmfem smooth 30", "method": "SmoothNMF", "params": {**default_params, **params_snmf, "lambda_L" : 30.0}},
    {"name": "snmfem smooth 3000", "method": "SmoothNMF", "params": {**default_params, **params_snmf, "lambda_L" : 300.0}},
]

In [ ]:
metrics = perform_simulations(samples, exp_list, params_evalution)

In [ ]:
metrics.shape

In [ ]:
print(print_results(exp_list, metrics))

# Run a single experiment and plotting results

In [ ]:
sample_num = 0

s = samples[sample_num]
Xflat, true_spectra, true_maps, G, shape_2d = load_data(s)

GPs = []
metrics = []
As = []
orders = []
losses = []

for exp in exp_list : 
    m, (GP, A), loss  = run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution,shape_2d)
    metric = m[:-1]
    order = m[-1]
    GPs.append(GP)
    As.append(A)
    metrics.append(metric)
    orders.append(order)
    losses.append(loss)
metrics = np.array(metrics)
orders = np.array(orders)

In [ ]:
# Ploting parameters
fontsize = 15
aspect_ratio = 3/4
scale = 20



In [ ]:
# # Test for the order
# from snmfem.measures import spectral_angle
# m2 = np.zeros([metrics.shape[0], metrics.shape[2]])
# for i in range(metrics.shape[0]):
#     for j in range(metrics.shape[2]):
#         ind = np.arange(k)[orders[i,0,j]]
#         m2[i,j] = spectral_angle(GPs[i][:,ind], true_spectra[j])
# np.testing.assert_allclose(m2,metrics[:,0,:])

In [ ]:
n_exp = len(exp_list)
fig, axes = plt.subplots(n_exp,k,figsize = (scale,scale/k * n_exp * aspect_ratio))
for i, exp in enumerate(exp_list): 
    for j in range(k):
        ind = np.arange(k)[orders[i,0,j]]
        axes[i,j].plot(true_spectra[j],'bo',label='truth',linewidth=4)
        axes[i,j].plot(GPs[i][:,ind] ,'r-',label='reconstructed',markersize=3.5)
        axes[i,j].set_title("{:.2f} deg".format(metrics[i,0,j]))

cols = ['Phase {}'.format(col) for col in range(k)]
rows = ['{}'.format(exp["name"]) for exp in exp_list]

for ax, col in zip(axes[0], cols):
    ax.set_title(col, fontsize=fontsize)

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)

fig.tight_layout()


In [ ]:
n_exp = len(exp_list)
fig, axes = plt.subplots(n_exp+1,k,figsize = (scale,scale/k * (n_exp+1) * aspect_ratio))
vmin = 0
vmax = np.max(true_maps)
cmap = plt.cm.gist_heat_r
for j in range(k):
    axes[0,j].imshow(true_maps[j].reshape(*shape_2d), vmin=vmin, vmax=vmax, cmap=cmap)
    for i, exp in enumerate(exp_list): 
        ind = np.arange(k)[orders[i,1,j]]
        axes[i+1,j].imshow(As[i][ind].reshape(*shape_2d), vmin=vmin, vmax=vmax, cmap=cmap)
        axes[i+1,j].set_title("Mse: {:.2f}".format(metrics[i,1,j]))
    
cols = ['MAP {}'.format(col) for col in range(k)]
rows = ["Ground truth"] + ['{}'.format(exp["name"]) for exp in exp_list]

for ax, col in zip(axes[0], cols):
    ax.set_title(col, fontsize=fontsize)

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)

fig.tight_layout()

# Quick and dirty way to look at the Loss

In [ ]:
indexes = np.arange(2, len(exp_list))
n_plot = len(indexes)

fig, axes = plt.subplots(n_plot, 2, figsize=(10, 4*n_plot))

for it, i in enumerate(indexes): 

    # plt.title(exp_list[i]["name"])

    axes[it, 0].plot(np.array(losses[i][1][0]), markersize=3.5)
    axes[it, 0].plot(losses[i][0],'r--',markersize=3.5)
    axes[it, 0].set_yscale("log")
    axes[it, 0].legend(losses[i][1][1] + ["Total"])
    axes[it, 0].set_xlabel("number of iterations")

    axes[it, 1].plot(losses[i][2], markersize=3.5)
    axes[it, 1].legend(["rel_A", "rel_P"])
    axes[it, 1].set_xlabel("number of iterations")

cols = ["Losses", "Evolution of A and P"]
rows = ['{}'.format(exp_list[i]["name"]) for i in indexes]

for ax, col in zip(axes[0], cols):
    ax.set_title(col, fontsize=fontsize)

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)

fig.tight_layout()